In [67]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [79]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text
soup = BeautifulSoup(data, 'html5lib')
table = soup.find('table')

In [80]:
file=[]
ncol=list(range(0,9))

for i in ncol :
    
    for row in table.find_all('tr'): # in html table row is represented by the table
    # Get all columns in each row.
        cols = row.find_all('p') # in html a column is represented by the tag td or p
        file.append(cols[i].getText())

    # convert to dataframe:
df= pd.DataFrame(file)
#df.head()

In [81]:
df["Postal Code"]=df[0].str[0:3]
df["Borough"]=df[0].str[3:].str.split("(",n=1,expand=True)[0]
df["Neigh"]=df[0].str.rsplit("(",n=2,expand=True)[1].str.split(")",n=1,expand=True)[0]
df["Neighbourhood"]=df["Neigh"].str.replace(" / ",",")
#df.head()

In [120]:
postal_code=df[(df["Borough"]!="Not assigned\n")]
postal_code=postal_code.drop([0,"Neigh"],axis=1)
postal_code.head()

,Postal Code,Borough,Neighbourhood
1,M1B,Scarborough,"Malvern,Rouge"
2,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
3,M1E,Scarborough,"Guildwood,Morningside,West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


time: 19.2 ms (started: 2022-06-25 04:32:49 +00:00)


In [121]:
postal_code.shape

(103, 3)

time: 4.8 ms (started: 2022-06-25 04:32:58 +00:00)


In [118]:
% pip install geocoder
% pip install --upgrade geocoder
% pip uninstall geocoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found existing installation: geocoder 1.38.1
Uninstalling geocoder-1.38.1:
  Would remove:
    /usr/local/bin/geocode
    /usr/local/lib/python3.7/dist-packages/geocoder-1.38.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/geocoder/*
Proceed (y/n)? y
  Successfully uninstalled geocoder-1.38.1
time: 3min 33s (started: 2022-06-25 04:27:12 +00:00)


In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

In [123]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!
time: 2.94 s (started: 2022-06-25 04:53:34 +00:00)


In [134]:
Geospatial_data = pd.read_csv('Geospatial_Coordinates.csv')
Geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


time: 16.9 ms (started: 2022-06-25 05:12:25 +00:00)


In [147]:
Coordinates=pd.merge(postal_code,Geospatial_data,on="Postal Code")
Coordinates.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


time: 19.4 ms (started: 2022-06-25 05:34:14 +00:00)


In [ ]:
from geopy.geocoders import Nominatim
! pip install folium==0.5.0
import folium # librería para graficar 

In [148]:
address = 'TORONTO,CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.
time: 293 ms (started: 2022-06-25 05:34:52 +00:00)


In [151]:
#T_data = Coordinates[Coordinates['Borough'] == 'Toronto'].reset_index(drop=True)
#T_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude


time: 13.7 ms (started: 2022-06-25 05:36:05 +00:00)


In [153]:
T_data = Coordinates[Coordinates['Borough'].str.contains('Toronto')].reset_index(drop=True)
T_data.groupby("Borough").size().sort_values(ascending=False)

Borough
Downtown Toronto                                                17
Central Toronto                                                  9
West Toronto                                                     6
East Toronto                                                     4
Downtown TorontoStn A PO Boxes25 The Esplanade                   1
East TorontoBusiness reply mail Processing Centre969 Eastern     1
East YorkEast Toronto                                            1
dtype: int64

time: 14.8 ms (started: 2022-06-25 05:36:15 +00:00)


In [154]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(T_data['Latitude'], T_data['Longitude'], T_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

time: 73.2 ms (started: 2022-06-25 05:36:18 +00:00)
